# Core (Always run)

In [ ]:
import shap
import mygene
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, label_binarize
from itertools import cycle
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, RFE, SelectFromModel
from sklearn.pipeline import Pipeline

Global Variables

In [ ]:
path_to_data = "data/"
dataset_file_name = "dataset.pq"

# Preprocess

## Load datasets

In [ ]:
df = pd.read_parquet(path_to_data + dataset_file_name)

## Cleaning

Transpose

In [ ]:
df = df.transpose()

print(f'Dataframe shape after transpose: {df.shape}')

df.head()

Apply subtypes

In [ ]:
excell_sheet_df = pd.read_excel('./assets/subtype_sheet.xlsx', sheet_name='RNA-Seq 1148')

for sample_id in df.index:
    print(f'Processing sample ID: {sample_id}')

    if sample_id in excell_sheet_df['Sample ID'].values:
        subtype = excell_sheet_df.loc[excell_sheet_df['Sample ID'] == sample_id, 'PAM50'].values[0]
        print(f'Subtype found: {subtype}')
        df.at[sample_id, 'Subtype'] = subtype

df.head()

Look for NaN

In [ ]:
if df.isna().sum().sum() > 0:
    print("Dataframe contains missing values. Dropping missing values.")
    print(f'Number of missing values: {df.isna().sum().sum()}')

    df = df.dropna()

    print("Missing values dropped.")
    print(f'Number of remaining missing values: {df.isna().sum().sum()}')
else:
    print("Dataframe does not contain missing values.")

# Exploratory Data Analysis (EDA)

Plot 1: Subtype distribution plot

More info about the subtypes in this paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC6985186/

In [ ]:
plot_df = df.copy()

plt.figure(figsize=(10, 6))
sns.countplot(data=plot_df, x='Subtype', order=plot_df['Subtype'].value_counts().index)
plt.title('Distribution of Subtypes')
plt.xlabel('Subtype')
plt.ylabel('Count')
plt.show()

Plot 2: Scatter plot

Observation: Contains a few outliers, not entirely sure what to do about them.

https://stats.stackexchange.com/questions/533503/when-should-you-remove-outliers-entire-dataset-or-train-dataset

In [ ]:
x_log_transformed = np.log1p(plot_df.select_dtypes(include=np.number))

scaler = StandardScaler()
df_scaled = scaler.fit_transform(x_log_transformed)

PCA_model = PCA(n_components=2)
pca_result = PCA_model.fit_transform(df_scaled)
plot_df['PCA1'] = pca_result[:, 0]
plot_df['PCA2'] = pca_result[:, 1]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=plot_df, x='PCA1', y='PCA2', hue='Subtype', palette='Set2')
plt.title('PCA Scatter Plot Colored by Subtype')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Subtype')
plt.show()

Plot 2.1: Scatter plot with outliers removed

In [ ]:
# Filter out outliers based on PCA1 and PCA2
filtered_plot_df = plot_df[plot_df['PCA1'] < 2000]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=filtered_plot_df, x='PCA1', y='PCA2', hue='Subtype', palette='Set2')
plt.title('PCA Scatter Plot with PCA1 < 2000 Colored by Subtype')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(title='Subtype')
plt.show()  

# Training

## Setup

Model training function

In [ ]:
def setup_pipeline(model, type: str, settings: dict = {}) -> Pipeline:
    """Returns pipeline based on model and type of feature selection. Feature selection types: rfe, sbm, skb"""

    base_list = [
        ('scaler', StandardScaler()),
        ('variance_threshold', VarianceThreshold(threshold=0.1))
    ]

    if type == 'rfe':
        return Pipeline(base_list + [
            ('feature_selection', RFE(estimator=LogisticRegression(max_iter=1500, random_state=42, class_weight='balanced'), n_features_to_select=50, step=0.1)),
            ('model', model)
        ])
    elif type == 'sbm':
        return Pipeline(base_list + [
            ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'), max_features=50)),
            ('model', model)
        ])
    elif type == 'skb':
        return Pipeline(base_list + [
            ('feature_selection', SelectKBest(score_func=f_classif, k=50)),
            ('model', model)
        ])
    else:
        raise ValueError("Invalid feature selection type. Choose from 'lr', 'rf', or 'kbest'.")

Labeling

In [ ]:
encoder = LabelEncoder()

y = encoder.fit_transform(df['Subtype'])
X = df.drop(columns=['Subtype'])

Normalization - log2

In [ ]:
X = np.log2(X + 1)

Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training Logistic Regression using LR, RF and KBest for feature selection

Logistic Regression - RFE(LogisticRegression) Feature Selection

In [ ]:
logreg_classifier_rfe = setup_pipeline(model=LogisticRegression(max_iter=1500, random_state=42), type='rfe').fit(X_train, y_train)
logreg_predictions_rfe = logreg_classifier_rfe.predict(X_test)

accuracy_logreg_rfe = accuracy_score(y_test, logreg_predictions_rfe)

print(f'Logistic Regression with RFE Feature Selection Accuracy: {accuracy_logreg_rfe:.4f}')

Logistic Regression - SelectBestModel(RandomForest) Feature Selection

In [ ]:
logreg_classifier_sbm = setup_pipeline(model=LogisticRegression(max_iter=1500, random_state=42), type='sbm').fit(X_train, y_train)
logreg_predictions_sbm = logreg_classifier_sbm.predict(X_test)

accuracy_logreg_sbm = accuracy_score(y_test, logreg_predictions_sbm)

print(f'Logistic Regression with SBM Feature Selection Accuracy: {accuracy_logreg_sbm:.4f}')

Logistic Regression - SelectKBest Feature Selection

In [ ]:
logreg_classifier_kbest = setup_pipeline(model=LogisticRegression(max_iter=1500, random_state=42), type='skb').fit(X_train, y_train)
logreg_predictions_kbest = logreg_classifier_kbest.predict(X_test)

accuracy_logreg_kbest = accuracy_score(y_test, logreg_predictions_kbest)

print(f'Logistic Regression with KBest Feature Selection Accuracy: {accuracy_logreg_kbest:.4f}')

## Training Random Forest using LR, RF and KBest for feature selection

Random Forest - RFE(LogisticRegression) Feature Selection

In [ ]:
rf_classifier_rfe = setup_pipeline(model=RandomForestClassifier(n_estimators=100, random_state=42), type='rfe').fit(X_train, y_train)
rf_predictions_rfe = rf_classifier_rfe.predict(X_test)

accuracy_rf_rfe = accuracy_score(y_test, rf_predictions_rfe)

print(f'Random Forest with RFE Feature Selection Accuracy: {accuracy_rf_rfe:.4f}')

Random Forest - SelectBestModel(RandomForest) Feature Selection

In [ ]:
rf_classifier_sbm = setup_pipeline(model=RandomForestClassifier(n_estimators=100, random_state=42), type='sbm').fit(X_train, y_train)
rf_predictions_sbm = rf_classifier_sbm.predict(X_test)

accuracy_rf_sbm = accuracy_score(y_test, rf_predictions_sbm)

print(f'Random Forest with SBM Feature Selection Accuracy: {accuracy_rf_sbm:.4f}')

Random Forest - SelectKBest Feature Selection

In [ ]:
rf_classifier_kbest = setup_pipeline(model=RandomForestClassifier(n_estimators=100, random_state=42), type='skb').fit(X_train, y_train)
rf_predictions_kbest = rf_classifier_kbest.predict(X_test)

accuracy_rf_kbest = accuracy_score(y_test, rf_predictions_kbest)

print(f'Random Forest with KBest Feature Selection Accuracy: {accuracy_rf_kbest:.4f}')

## Training XGBoost using LR, RF and KBest for feature selection

XGBoost - RFE(LogisticRegression) Feature Selection

In [ ]:
xgb_classifier_rfe = setup_pipeline(model=XGBClassifier(eval_metric='mlogloss', random_state=42), type='rfe').fit(X_train, y_train)
xgb_predictions_rfe = xgb_classifier_rfe.predict(X_test)

accuracy_xgb_rfe = accuracy_score(y_test, xgb_predictions_rfe)

print(f'XGBoost with RFE Feature Selection Accuracy: {accuracy_xgb_rfe:.4f}')


XGBoost - SelectBestModel(RandomForest) Feature Selection

In [ ]:
xgb_classifier_sbm = setup_pipeline(model=XGBClassifier(eval_metric='mlogloss', random_state=42), type='sbm').fit(X_train, y_train)
xgb_predictions_sbm = xgb_classifier_sbm.predict(X_test)

accuracy_xgb_sbm = accuracy_score(y_test, xgb_predictions_sbm)

print(f'XGBoost with SBM Feature Selection Accuracy: {accuracy_xgb_sbm:.4f}')

XGBoost - SelectKBest Feature Selection

In [ ]:
xgb_classifier_skbest = setup_pipeline(model=XGBClassifier(eval_metric='mlogloss', random_state=42), type='skb').fit(X_train, y_train)
xgb_predictions_kbest = xgb_classifier_skbest.predict(X_test)

accuracy_xgb_kbest = accuracy_score(y_test, xgb_predictions_kbest)

print(f'XGBoost with SKB Feature Selection Accuracy: {accuracy_xgb_kbest:.4f}')

# TRAINING LEGACY

Scaler

In [ ]:
# standard_scaler = StandardScaler()
# X_train_scaled = standard_scaler.fit_transform(X_train)
# X_test_scaled = standard_scaler.transform(X_test)

Variance Threshold

In [ ]:
# variance_filter = VarianceThreshold(threshold=0.1)
# X_train_filtered = variance_filter.fit_transform(X_train_scaled)
# X_test_filtered = variance_filter.transform(X_test_scaled)

Feature selection (SelectKBest)

In [ ]:
# SelectKBest_model = SelectKBest(score_func=f_classif, k=50)
# X_train_selected = SelectKBest_model.fit_transform(X_train_filtered, y_train)
# X_test_selected = SelectKBest_model.transform(X_test_filtered)

Random Forest

In [ ]:
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train_selected, y_train)

# y_pred_rf = rf_classifier.predict(X_test_selected)

# accuracy_rf = accuracy_score(y_test, y_pred_rf)

# print(f'Random Forest Classifier Accuracy: {accuracy_rf:.6f}')

Logistic Regression

In [ ]:
# logreg_classifier = LogisticRegression(max_iter=1500, random_state=42)
# logreg_classifier.fit(X_train_selected, y_train)

# y_pred_lr = logreg_classifier.predict(X_test_selected)

# accuracy_lr = accuracy_score(y_test, y_pred_lr)

# print(f'Logistic Regression Classifier Accuracy: {accuracy_lr:.6f}')

XGBoost

Parameters for XGBClassifier: https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py

In [ ]:
# xgb_classifier = XGBClassifier(
#     tree_method='auto',
#     n_estimators=100,
#     eval_metric='mlogloss', 
#     random_state=42,
#     max_depth=6,
# )
# xgb_classifier.fit(X_train_selected, y_train)

# y_pred_xgb = xgb_classifier.predict(X_test_selected)

# accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

# print(f'XGBoost Classifier Accuracy: {accuracy_xgb:.6f}')

# Training Results Analysis

## Classification Report and Confusion Matrix

### Setup

In [ ]:
def generate_classification_report(y_true, y_pred):
    report = classification_report(
        y_true,
        y_pred
    )
    print("Classification Report:")
    print(report)

def generate_confusion_matrix(y_true, y_pred, class_names, title_additional=""):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    if title_additional:
        plt.title(f'Confusion Matrix - {title_additional}')
    else:
        plt.title('Confusion Matrix')
    plt.show()

def plot_cm_on_ax(ax, y_true, y_pred, class_names, title=""):
    cm = confusion_matrix(y_true, y_pred)
    
    sns.heatmap(cm, 
                annot=True, 
                fmt='d', 
                cmap='Blues', 
                xticklabels=class_names, 
                yticklabels=class_names,
                ax=ax,          # <<< Plots on the provided subplot
                cbar=True)      # You can set this to False if you want
    
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(title, fontsize=14)

### Classification Reports - Logistic Regression

In [ ]:
all_titles = ['Model (LR-RFE)', 'Model (RF-Select)', 'Model (KBest)']

In [ ]:
all_preds_lr = [logreg_predictions_rfe, logreg_predictions_sbm, logreg_predictions_kbest]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 6))

for ax, y_pred, title in zip(axes, all_preds_lr, all_titles):
    plot_cm_on_ax(ax, 
                  y_true=y_test, 
                  y_pred=y_pred, 
                  class_names=encoder.classes_, 
                  title=title
                )

fig.suptitle('Logistic Regression Model - Feature Selection', fontsize=20, y=1.05)
plt.tight_layout()
plt.show()

### Classification Reports - Random Forest

In [ ]:
all_preds_rf = [rf_predictions_rfe, rf_predictions_sbm, rf_predictions_kbest]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 6))

for ax, y_pred, title in zip(axes, all_preds_rf, all_titles):
    plot_cm_on_ax(ax, 
                  y_true=y_test, 
                  y_pred=y_pred, 
                  class_names=encoder.classes_, 
                  title=title
                )

fig.suptitle('Random Forest Model - Feature Selection', fontsize=20, y=1.05)
plt.tight_layout()
plt.show()

### Classification Reports - XGBoost

In [ ]:
all_preds_xgb = [xgb_predictions_rfe, xgb_predictions_sbm, xgb_predictions_kbest]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(24, 6))
for ax, y_pred, title in zip(axes, all_preds_xgb, all_titles):
    plot_cm_on_ax(ax, 
                  y_true=y_test, 
                  y_pred=y_pred, 
                  class_names=encoder.classes_, 
                  title=title
                )

fig.suptitle('XGBoost Model - Feature Selection', fontsize=20, y=1.05)
plt.tight_layout()
plt.show()

## Train ACC vs Test ACC

Setup

In [ ]:
def train_test_acc_compare(model, X_train, y_train, y_test, y_pred):
    acc_score = accuracy_score(y_test, y_pred)
    train_pred = model.predict(X_train)
    train_f1 = f1_score(y_train, train_pred, average='weighted')
    test_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f'Model name: {model.named_steps["model"].__class__.__name__}')
    print(f'Feature Selection Method: {model.named_steps["feature_selection"].__class__.__name__}')
    print('---')
    print(f'Accuracy Score: {acc_score:.4f}')
    print('---')
    print(f'Training F1 Score: {train_f1:.4f}')
    print(f'Testing F1 Score: {test_f1:.4f}')
    print(f'F1 Score Difference (Train - Test): {train_f1 - test_f1:.4f}')

Logistical Regression

In [ ]:
train_test_acc_compare(logreg_classifier_rfe, X_train, y_train, y_test, logreg_predictions_rfe)
print("\n")
train_test_acc_compare(logreg_classifier_sbm, X_train, y_train, y_test, logreg_predictions_sbm)
print("\n")
train_test_acc_compare(logreg_classifier_kbest, X_train, y_train, y_test, logreg_predictions_kbest)

Random Forest

In [ ]:
train_test_acc_compare(rf_classifier_rfe, X_train, y_train, y_test, rf_predictions_rfe)
print("\n")
train_test_acc_compare(rf_classifier_sbm, X_train, y_train, y_test, rf_predictions_sbm)
print("\n")
train_test_acc_compare(rf_classifier_kbest, X_train, y_train, y_test, rf_predictions_kbest)

XGBoost

In [ ]:
train_test_acc_compare(xgb_classifier_rfe, X_train, y_train, y_test, xgb_predictions_rfe)
print("\n")
train_test_acc_compare(xgb_classifier_sbm, X_train, y_train, y_test, xgb_predictions_sbm)
print("\n")
train_test_acc_compare(xgb_classifier_skbest, X_train, y_train, y_test, xgb_predictions_kbest)

# Hyperparameter Tuning

## Tuning process

Possible scoring values are: 'accuracy', 'f1_macro', 'f1_weighted'

In [ ]:
scoring = 'f1_macro' # metric for evaluation
n_features = [50, 75, 100]
n_jobs = 6

In [ ]:
logreg_sbm_pipeline = setup_pipeline(model=LogisticRegression(max_iter=7500, random_state=42), type='sbm')

param_grid_logreg_sbm = [
    {
        'feature_selection__max_features': n_features,
        'model__penalty': ['l1'],
        'model__solver': ['saga'],
        'model__C': [0.001, 0.01, 0.1, 1, 10],
        'model__class_weight': ['balanced']
    },
    {
        'feature_selection__max_features': n_features,
        'model__penalty': ['l2'],
        'model__solver': ['lbfgs', 'newton-cg', 'newton-cholesky'],
        'model__C': [0.001, 0.01, 0.1, 1, 10],
        'model__class_weight': ['balanced']
    }
]
grid_search_logreg_sbm = GridSearchCV(estimator=logreg_sbm_pipeline, param_grid=param_grid_logreg_sbm, scoring=scoring, cv=5, n_jobs=n_jobs, verbose=2)

grid_search_logreg_sbm.fit(X_train, y_train)

print(f'Best parameters for Logistic Regression with SBM: {grid_search_logreg_sbm.best_params_}')
print(f'Best {scoring} score for Logistic Regression with SBM: {grid_search_logreg_sbm.best_score_:.4f}')

In [ ]:
tuned_logreg_sbm_model = grid_search_logreg_sbm.best_estimator_['model']

print(f'Tuned Logistic Regression Model: {tuned_logreg_sbm_model}')

main_classifier = setup_pipeline(model=tuned_logreg_sbm_model, type='sbm').fit(X_train, y_train)
main_predictions = main_classifier.predict(X_test)

accuracy_main = accuracy_score(y_test, main_predictions)

print(f'Main Classifier Accuracy after Tuning: {accuracy_main:.4f}')

# Hyperparameter Tuning Analysis

## Train ACC vs Test ACC

In [ ]:
train_test_acc_compare(main_classifier, X_train, y_train, y_test, main_predictions)

## SHAP Analysis

Setup

# Classification Reports (LEGACY)

Classification Report - LR + LR Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_lr),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_lr),
#     class_names=encoder.classes_
# )

Classification Report - LR + RF Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_rf),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_rf),
#     class_names=encoder.classes_
# )

Classification Report - LR + KBest Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_kbest),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(logreg_predictions_kbest),
#     class_names=encoder.classes_
# )

### Classification Reports for RF Classifier with LR, RF and KBest as selection methods

Classification Report - RF + LR Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_lr),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_lr),
#     class_names=encoder.classes_
# )

Classification Report - RF + RF Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_rf),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_rf),
#     class_names=encoder.classes_
# )

Classification Report - RF + KBest Selection

In [ ]:
# generate_classification_report(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_kbest),
# )

# generate_confusion_matrix(
#     encoder.inverse_transform(y_test),
#     encoder.inverse_transform(rf_predictions_kbest),
#     class_names=encoder.classes_
# )

# REPORT (LEGACY)

In [ ]:
# print("Classification Report for Random Forest:")
# print(classification_report(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_rf)))

In [ ]:
# cm = confusion_matrix(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_rf))
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
# plt.title('Confusion Matrix for Random Forest Classifier')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

In [ ]:
# print("Classification Report for Logistic Regression:")
# print(classification_report(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_lr)))

In [ ]:
# cm = confusion_matrix(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_lr))
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
# plt.title('Confusion Matrix for Logistic Regression Classifier')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

In [ ]:
# print("Classification Report for XGBoost Classifier:")
# print(classification_report(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_xgb)))

In [ ]:
# cm = confusion_matrix(encoder.inverse_transform(y_test), encoder.inverse_transform(y_pred_xgb))
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=encoder.classes_, yticklabels=encoder.classes_)
# plt.title('Confusion Matrix for XGBoost Classifier')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

## ROC Curve

In [ ]:
# classes = range(len(encoder.classes_))
# class_labels = encoder.classes_
# number_of_classes = len(class_labels)

Random Forest

In [ ]:
# y_pred_proba_rf = rf_classifier.predict_proba(X_test_selected)
# y_test_binarized = label_binarize(y_test, classes=classes)

# plt.figure(figsize=(10, 8))
# colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])

# for i, color in zip(range(number_of_classes), colors):
#     labels_for_class = y_test_binarized[:, i]
#     probs_for_class = y_pred_proba_rf[:, i]

#     fpr, tpr, _ = roc_curve(labels_for_class, probs_for_class)
#     roc_auc = auc(fpr, tpr)

#     plt.plot(fpr, tpr, color=color, lw=2, label=f'ROC curve of class {class_labels[i]} (area = {roc_auc:.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curves for Random Forest Classifier')
# plt.legend(loc='lower right')
# plt.grid(True)
# plt.show()

Logistic Regression

In [ ]:
# y_pred_proba_lr = logreg_classifier.predict_proba(X_test_selected)
# y_test_binarized = label_binarize(y_test, classes=classes)

# plt.figure(figsize=(10, 8))
# colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])

# for i, color in zip(range(number_of_classes), colors):
#     labels_for_class = y_test_binarized[:, i]
#     probs_for_class = y_pred_proba_lr[:, i]

#     fpr, tpr, _ = roc_curve(labels_for_class, probs_for_class)
#     roc_auc = auc(fpr, tpr)

#     plt.plot(fpr, tpr, color=color, lw=2, label=f'ROC curve of class {class_labels[i]} (area = {roc_auc:.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curves for Logistic Regression Classifier')
# plt.legend(loc='lower right')
# plt.grid(True)
# plt.show()

XGBoost

In [ ]:
# y_pred_proba_xgb = xgb_classifier.predict_proba(X_test_selected)
# y_test_binarized = label_binarize(y_test, classes=classes)

# plt.figure(figsize=(10, 8))
# colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])

# for i, color in zip(range(number_of_classes), colors):
#     labels_for_class = y_test_binarized[:, i]
#     probs_for_class = y_pred_proba_xgb[:, i]

#     fpr, tpr, _ = roc_curve(labels_for_class, probs_for_class)
#     roc_auc = auc(fpr, tpr)

#     plt.plot(fpr, tpr, color=color, lw=2, label=f'ROC curve of class {class_labels[i]} (area = {roc_auc:.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curves for XGBoost Classifier')
# plt.legend(loc='lower right')
# plt.grid(True)
# plt.show()

## Train ACC vs Test ACC

Random Forest

In [ ]:
# train_pred_rf = rf_classifier.predict(X_train_selected)
# train_f1_rf = f1_score(y_train, train_pred_rf, average='weighted')
# test_f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

# print(f'Training F1 Score for Random Forest: {train_f1_rf:.6f}')
# print(f'Test F1 Score for Random Forest: {test_f1_rf:.6f}')
# print(f'F1 Score Difference for Random Forest: {train_f1_rf - test_f1_rf:.6f}')

Logistical Regression

In [ ]:
# train_pred_lr = logreg_classifier.predict(X_train_selected)
# train_f1_lr = f1_score(y_train, train_pred_lr, average='weighted')
# test_f1_lr = f1_score(y_test, y_pred_lr, average='weighted')

# print(f'Training F1 Score for Logistic Regression: {train_f1_lr:.6f}')
# print(f'Test F1 Score for Logistic Regression: {test_f1_lr:.6f}')
# print(f'F1 Score Difference for Logistic Regression: {train_f1_lr - test_f1_lr:.6f}')

XGBoost

In [ ]:
# train_pred_xgb = xgb_classifier.predict(X_train_selected)
# train_f1_xgb = f1_score(y_train, train_pred_xgb, average='weighted')
# test_f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')

# print(f'Training F1 Score for XGBoost: {train_f1_xgb:.6f}')
# print(f'Test F1 Score for XGBoost: {test_f1_xgb:.6f}')
# print(f'F1 Score Difference for XGBoost: {train_f1_xgb - test_f1_xgb:.6f}')

# Hyperparameter Tuning (LEGACY)

## GridsearchCV

Global variables

In [ ]:
# core_usage = 6 # -1 = ALL Cores
# scoring = "f1_macro"
# select_k_list = [50, 75, 100]

Define global pipeline

In [ ]:
# preprocessing_steps = [
#     ('scaler', StandardScaler()),
#     ('filter', VarianceThreshold(threshold=0.1)),
#     ('selector', SelectKBest(score_func=f_classif)),
# ]

Logistical Regression

In [ ]:
# max_iteration = 5000
# random_state = 42

# pipeline_lr = Pipeline(preprocessing_steps + [
#     ('model', LogisticRegression(max_iter=max_iteration, random_state=random_state))
# ])

# param_grid_lr = {
#     'selector__k': select_k_list,
#     'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
#     'model__solver': ['lbfgs', 'sag', 'saga'],
#     'model__class_weight': ['balanced', None]
# }

# grid_search_lr = GridSearchCV(
#     estimator=pipeline_lr,
#     param_grid=param_grid_lr,
#     scoring=scoring,
#     cv=5,
#     n_jobs=core_usage,
#     verbose=2
# )

# grid_search_lr.fit(X_train, y_train)

# print(f'Best parameters found: {grid_search_lr.best_params_}')
# print(f'Best {scoring} score: {grid_search_lr.best_score_:.6f}')

## GridsearchCV Analysis Result

Logistical Regression

In [ ]:
# SelectKBest_model = SelectKBest(score_func=f_classif, k=grid_search_lr.best_params_['selector__k'])
# X_train_selected = SelectKBest_model.fit_transform(X_train_filtered, y_train)
# X_test_selected = SelectKBest_model.transform(X_test_filtered)

# logreg_classifier = LogisticRegression(
#     max_iter=max_iteration, 
#     random_state=random_state,
#     C=grid_search_lr.best_params_['model__C'],
#     solver=grid_search_lr.best_params_['model__solver'],
#     class_weight=grid_search_lr.best_params_['model__class_weight']
# )

# logreg_classifier.fit(X_train_selected, y_train)

# y_pred_lr = logreg_classifier.predict(X_test_selected)

# accuracy_lr = accuracy_score(y_test, y_pred_lr)

# print(f'Logistic Regression Classifier Accuracy: {accuracy_lr:.6f}')

# train_pred_lr = logreg_classifier.predict(X_train_selected)
# train_f1_lr = f1_score(y_train, train_pred_lr, average='macro')
# test_f1_lr = f1_score(y_test, y_pred_lr, average='macro')

# print(f'Training F1 Score for Logistic Regression: {train_f1_lr:.6f}')
# print(f'Test F1 Score for Logistic Regression: {test_f1_lr:.6f}')
# print(f'F1 Score Difference for Logistic Regression: {train_f1_lr - test_f1_lr:.6f}')

# SHAP Analysis

Setup

In [ ]:
# all_gene_names = list(df.columns)

# best_model = grid_search_lr.best_estimator_

# selector = best_model.named_steps['selector']
# lr_model = best_model.named_steps['model']
# selected_indices = selector.get_support(indices=True)

# selected_gene_names = [all_gene_names[i] for i in selected_indices]
# print(f"Selected {len(selected_gene_names)} genes")

# # Print first 10 selected gene names for verification
# print(selected_gene_names[:10]) 

# selected_gene_names = [gene.split('.')[0] for gene in selected_gene_names]

# # Print first 10 selected gene names after removing version numbers for verification
# print(f"Selected gene names after removing version numbers: {selected_gene_names[:10]}")

# # Gene symbol mapping using mygene
# mg = mygene.MyGeneInfo()
# gene_info = mg.querymany(
#     selected_gene_names,
#     scopes='ensembl.gene',
#     fields='symbol',
#     species='human',
#     verbose=False
# )

# # Print first 5 entries of gene_info for verification
# print(gene_info[:5])

# ensembl_to_symbol = {}
# for result in gene_info:
#     ensembl_id = result['query']
#     gene_symbol = result.get('symbol', ensembl_id)
#     ensembl_to_symbol[ensembl_id] = gene_symbol

# gene_symbols = [ensembl_to_symbol.get(gene, gene) for gene in selected_gene_names]

# class_names = ['Basal', 'Her2', 'LumA', 'LumB', 'Normal']

Explainer

In [ ]:
# explainer = shap.LinearExplainer(lr_model, X_train_selected)
# shap_values = explainer.shap_values(X_test_selected)

Bar plot

In [ ]:
# plt.figure(figsize=(14, 10))
# shap.summary_plot(
#     shap_values,
#     X_test_selected,
#     feature_names=gene_symbols,
#     class_names=class_names,
#     plot_type="bar"
# )
# plt.tight_layout()
# plt.show()

Top Genes per Subtype

In [ ]:
# for class_idx, class_name in enumerate(class_names):
#     # Calculate mean absolute SHAP value per gene for this class
#     mean_abs_shap = np.abs(shap_values[class_idx]).mean(axis=0)
    
#     # Get top 10
#     top_indices = np.argsort(mean_abs_shap)[-10:][::-1]
    
#     print(f"\n{class_name.upper()}:")
#     print("-" * 70)
#     for rank, idx in enumerate(top_indices, 1):
#         gene_symbol = gene_symbols[idx]
#         ensembl_id = selected_gene_names[idx]
#         importance = mean_abs_shap[idx]
#         print(f"  {rank:2d}. {gene_symbol:15s} ({ensembl_id}) - Importance: {importance:.4f}")

# Alternative selection method: Random Forest Selection (LEGACY)

In [ ]:
# y = df['Subtype']
# X = df.drop(columns=['Subtype'])

In [ ]:
# X = np.log2(X + 1)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

Random Forest Selection

In [ ]:
# rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=6)

# rf_selector.fit(X_train_scaled, y_train)

# importances = rf_selector.feature_importances_
# top_n_genes = np.argsort(importances)[-50:]

# X_train_selected = X_train_scaled[:, top_n_genes]
# X_test_selected = X_test_scaled[:, top_n_genes]

Logistic Regression Selection

In [ ]:
# lr_classifier = LogisticRegression(max_iter=1500, random_state=42, class_weight='balanced')

# rfe_selector = RFE(estimator=lr_classifier, n_features_to_select=50, step=0.1, verbose=1)
# rfe_selector.fit(X_train_scaled, y_train)

# selected_indices_lr = np.where(rfe_selector.support_)[0]
# print(f"Selected features for LR:\n{selected_indices_lr}\n")

# top_n_genes = selected_indices_lr

# X_train_selected = X_train_scaled[:, top_n_genes]
# X_test_selected = X_test_scaled[:, top_n_genes]

List selected genes

In [ ]:
# gene_ids = X.columns[top_n_genes]

# gene_ids = [gene.split('.')[0] for gene in gene_ids]

# mg = mygene.MyGeneInfo()
# mygene_info = mg.querymany(
#     list(gene_ids),
#     scopes='ensembl.gene',
#     fields='symbol',
#     species='human',
#     verbose=False
# )

# print("All genes retrieved from mygene:")
# for gene in mygene_info:
#     print(f"Gene ID: {gene['query']}, Symbol: {gene['symbol'] if 'symbol' in gene else 'N/A'}")

Logistic Regression

In [ ]:
# logreg_classifier = LogisticRegression(max_iter=1500, random_state=42, class_weight='balanced')
# logreg_classifier.fit(X_train_selected, y_train)

# y_pred_lr = logreg_classifier.predict(X_test_selected)

# accuracy_lr = accuracy_score(y_test, y_pred_lr)

# print(f'Logistic Regression Classifier Accuracy with Top 100 RF Genes: {accuracy_lr:.6f}')

# # F1 Score to check for overfitting
# train_pred = logreg_classifier.predict(X_train_selected)
# train_f1 = f1_score(y_train, train_pred, average='macro')
# test_f1 = f1_score(y_test, y_pred_lr, average='macro')

# print(f'Training F1 Score for Logistic Regression with Top 100 RF Genes: {train_f1:.6f}')
# print(f'Test F1 Score for Logistic Regression with Top 100 RF Genes: {test_f1:.6f}')
# print(f'F1 Score Difference for Logistic Regression with Top 100 RF Genes: {train_f1 - test_f1:.6f}')

Random Forest

In [ ]:
# random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# random_forest_classifier.fit(X_train_selected, y_train)

# y_pred_rf = random_forest_classifier.predict(X_test_selected)

# accuracy_rf = accuracy_score(y_test, y_pred_rf)

# print(f'Random Forest Classifier Accuracy with Top 100 RF Genes: {accuracy_rf:.6f}')

# # F1 Score to check for overfitting
# train_pred_rf = random_forest_classifier.predict(X_train_selected)
# train_f1_rf = f1_score(y_train, train_pred_rf, average='macro')
# test_f1_rf = f1_score(y_test, y_pred_rf, average='macro')

# print(f'Training F1 Score for Random Forest with Top 100 RF Genes: {train_f1_rf:.6f}')
# print(f'Test F1 Score for Random Forest with Top 100 RF Genes: {test_f1_rf:.6f}')
# print(f'F1 Score Difference for Random Forest with Top 100 RF Genes: {train_f1_rf - test_f1_rf:.6f}')

XGBoost

In [ ]:
# encoder = LabelEncoder()

# y = encoder.fit_transform(df['Subtype'])
# X = df.drop(columns=['Subtype'])

# X = np.log2(X + 1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=6)

# rf_selector.fit(X_train_scaled, y_train)

# importances = rf_selector.feature_importances_
# top_100_genes = np.argsort(importances)[-50:]

# X_train_selected = X_train_scaled[:, top_100_genes]
# X_test_selected = X_test_scaled[:, top_100_genes]

# xgb_classifier = XGBClassifier(
#     tree_method='auto',
#     n_estimators=100,
#     eval_metric='mlogloss', 
#     random_state=42,
#     max_depth=6,
# )
# xgb_classifier.fit(X_train_selected, y_train)

# y_pred_xgb = xgb_classifier.predict(X_test_selected)

# accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

# print(f'XGBoost Classifier Accuracy with Top 100 RF Genes: {accuracy_xgb:.6f}')

# # F1 Score to check for overfitting
# train_pred_xgb = xgb_classifier.predict(X_train_selected)
# train_f1_xgb = f1_score(y_train, train_pred_xgb, average='macro')
# test_f1_xgb = f1_score(y_test, y_pred_xgb, average='macro')

# print(f'Training F1 Score for XGBoost with Top 100 RF Genes: {train_f1_xgb:.6f}')
# print(f'Test F1 Score for XGBoost with Top 100 RF Genes: {test_f1_xgb:.6f}')
# print(f'F1 Score Difference for XGBoost with Top 100 RF Genes: {train_f1_xgb - test_f1_xgb:.6f}')